In [3]:
import effis.composition
import logging
import os

effis.composition.log.CompositionLogger.SetLevel(logging.DEBUG)
Example = os.path.join(effis.composition.ExamplesPath, "NodeShare-Hostname-Sleep", "HostnameSleep.py")

where = "perlmutter"

if where == "crusher":
    w = effis.composition.Workflow(Name="ShareTest", Charge="phy122", Machine="Crusher", TimeIndex=False)
    w.SchedulerDirectives += "--core-spec=0"
elif where == "perlmutter":
    w = effis.composition.Workflow(Name="ShareTest", Charge="m4564",  TimeIndex=False, Machine="perlmutter_cpu", Node=effis.composition.Node(cores=128, gpus=0))
    w.SchedulerDirectives += "--constraint=cpu"
    w.SchedulerDirectives += "--time=00:05:00"


w.Input += "testdir"
w.Input += effis.composition.Input("TestFile02.txt", link=True)

App1 = effis.composition.Application(Filepath=Example, Ranks=2, RanksPerNode=1, CoresPerRank=8, GPUsPerRank=0, Name="host1")
App1.CommandLineArguments += "--SleepTime=30"
App1.ShareKey = "Basic"
App1.Input += effis.composition.Input("TestFile01.txt", rename="Other.txt")
App1.Input += effis.composition.Input("TestFile02.txt", rename="Other2.txt", outpath="NewSubdir")

App2 = effis.composition.Application(Filepath=Example, Ranks=2, RanksPerNode=1, CoresPerRank=56, GPUsPerRank=0, Name="host2")
App2.CommandLineArguments += "--SleepTime=60"
App2.ShareKey = "Basic"
App2.Input += effis.composition.Input("testdir", rename="setupdir", outpath="other")

w += App1 + App2

w.Create()

print(w.Directory, w.Applications[0].Directory)

2024-04-09 02:54:26.009 - DEBUG - Node 0 cpus --> 1-8: host1:0, 9-64: host2:0
2024-04-09 02:54:26.009 - DEBUG - Node 0 gpus --> []


Runs/ShareTest/esuchyta/EFFIS/run-0.iteration-0 Runs/ShareTest/esuchyta/EFFIS/run-0.iteration-0/host1
